### Experiment with MongoEngine

The ODM we use to access MongoDB

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document

from datetime import date

Open the database:

In [2]:
DB.open('pytest')

Make an account:

In [3]:
acct = Account(name='equity', category='Q')

Save it:

In [4]:
acct.save()

<Account: <Acct equity Q>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [5]:
Account.objects

[<Account: <Acct equity Q>, <Account: <Acct yoyodyne I>, <Account: <Acct checking A>, <Account: <Acct amex L>, <Account: <Acct visa L>, <Account: <Acct groceries E>, <Account: <Acct household E>, <Account: <Acct mortgage E>, <Account: <Acct car E>, <Account: <Acct travel E>, <Account: <Acct equity Q>, <Account: <Acct equity Q>, <Account: <Acct equity Q>]

In [6]:
Account.objects[0]

<Account: <Acct equity Q>

In [7]:
acct = Account.objects[0]

In [8]:
acct.name

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [9]:
DB.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [10]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest')

In [11]:
db = DB.database

In [12]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [13]:
db.account.find_one()

{'_id': ObjectId('67e41dd9a0fded77104fa31d'),
 'name': 'equity',
 'category': 'Q'}

In [14]:
for c in db.list_collections():
    print(c)

{'name': 'foo', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\n\x86C\xa2\xabQE\xcf\x9a\xff\x0c/\xa6\xd8\xdd\xe4', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'foo_bar', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\x15\xd9 \x96\x05\xdc@\xa3\x81\xe5\xde\xaa2\xa2;+', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'transaction', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'y\xf5x\xbaf\xa2H\xc2\xa0\xe4t\xf6\xd7\xbe\x08D', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'entry', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\x95)\x15#\xb8\n@\x8d\xa8\x8b\x14v\xc9\xe2\x9e\xb2', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'account', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xa9\xc8I/\xef]D-\x918\x85\xba\t\xde\xab\

In [15]:
for name in db.list_collection_names():
    print(name)

foo
foo_bar
transaction
entry
account
bar


In [16]:
db['account']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [17]:
db['account'].find_one()

{'_id': ObjectId('67e41dd9a0fded77104fa31d'),
 'name': 'equity',
 'category': 'Q'}

In [18]:
for obj in db['account'].find():
    print(obj)

{'_id': ObjectId('67e41dd9a0fded77104fa31d'), 'name': 'equity', 'category': 'Q'}
{'_id': ObjectId('67e41dd9a0fded77104fa31e'), 'name': 'yoyodyne', 'category': 'I'}
{'_id': ObjectId('67e41dd9a0fded77104fa31f'), 'name': 'checking', 'category': 'A'}
{'_id': ObjectId('67e41dd9a0fded77104fa320'), 'name': 'amex', 'category': 'L'}
{'_id': ObjectId('67e41dd9a0fded77104fa321'), 'name': 'visa', 'category': 'L'}
{'_id': ObjectId('67e41dd9a0fded77104fa322'), 'name': 'groceries', 'category': 'E'}
{'_id': ObjectId('67e41dd9a0fded77104fa323'), 'name': 'household', 'category': 'E'}
{'_id': ObjectId('67e41dd9a0fded77104fa324'), 'name': 'mortgage', 'category': 'E'}
{'_id': ObjectId('67e41dd9a0fded77104fa325'), 'name': 'car', 'category': 'E'}
{'_id': ObjectId('67e41dd9a0fded77104fa326'), 'name': 'travel', 'category': 'E'}
{'_id': ObjectId('67e41ddfaf710d6e27750ce6'), 'name': 'equity', 'category': 'Q'}
{'_id': ObjectId('67e41e8ca41d5256f8638f84'), 'name': 'equity', 'category': 'Q'}
{'_id': ObjectId('67e41

### From Low Level to High Level

Question:  given a collection name ("account") can we find the corresponding MongoEngine class (Account)?

In [19]:
Document

mongoengine.document.Document

In [20]:
Document.__subclasses__()

[mongoengine.document.DynamicDocument,
 dexter.DB.Account,
 dexter.DB.Entry,
 dexter.DB.Transaction]

In [21]:
[cls for cls in Document.__subclasses__() if hasattr(cls, 'objects')]

[dexter.DB.Account, dexter.DB.Entry, dexter.DB.Transaction]

In [22]:
Account._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'account',
 'index_specs': []}

In [23]:
for cls in Document.__subclasses__():
    if not hasattr(cls, 'objects'):
        continue
    print(cls._meta['collection'], cls)

account <class 'dexter.DB.Account'>
entry <class 'dexter.DB.Entry'>
transaction <class 'dexter.DB.Transaction'>


### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [24]:
t = Transaction(description='hi', comment='aloha')

In [25]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [26]:
t.tags

[]

In [27]:
t.entries

[]

### Entries

In [28]:
e = Entry(uid='xxx', column='credit', date='2025-03-05', amount=1000, account='unknown')

In [29]:
type(e)

dexter.DB.Entry

In [30]:
e

<Entry: <En 2025-03-05 unknown -$1000.0>>

In [31]:
e.column

<Column.cr: 'credit'>

In [32]:
e.amount

1000.0

In [33]:
e.hash

'9ea47c3f44dca9cfe40a343f2c1bf5c5'

In [34]:
len(e.hash)

32

### References

The big test -- can we add that Entry to the transaction?

In [35]:
t.entries.append(p)

In [36]:
t.entries

[<Entry: <En 2025-03-05 unknown -$1000.0>>]

Yes!  🎉

### Misc Commands

In [37]:
db.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')

In [38]:
db.stats.find_one

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')>

In [39]:
db.list_collection_names()

['foo', 'foo_bar', 'transaction', 'entry', 'account', 'bar']

In [40]:
db.command('count','account')

{'n': 12, 'ok': 1.0}

In [41]:
db.command('hello')

{'isWritablePrimary': True,
 'topologyVersion': {'processId': ObjectId('67c4fbece8eb6b0fbfe3b917'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2025, 3, 26, 15, 34, 36, 507000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 1052,
 'minWireVersion': 0,
 'maxWireVersion': 25,
 'readOnly': False,
 'ok': 1.0}

In [42]:
db.command('hostInfo')

{'system': {'currentTime': datetime.datetime(2025, 3, 26, 15, 34, 36, 512000),
  'hostname': 'cthulhu.local',
  'cpuAddrSize': 64,
  'memSizeMB': 65536,
  'memLimitMB': 65536,
  'numCores': 12,
  'numCoresAvailableToProcess': 12,
  'numPhysicalCores': 12,
  'numCpuSockets': 1,
  'cpuArch': 'arm64',
  'numaEnabled': False,
  'numNumaNodes': 1},
 'os': {'type': 'Darwin', 'name': 'Mac OS X', 'version': '24.3.0'},
 'extra': {'versionString': 'Darwin Kernel Version 24.3.0: Thu Jan  2 20:24:23 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T6020',
  'alwaysFullSync': 0,
  'nfsAsync': 0,
  'model': 'Mac14,13',
  'cpuString': 'Apple M2 Max',
  'pageSize': 16384,
  'scheduler': 'edge'},
 'ok': 1.0}

In [43]:
db.command('ping')

{'ok': 1.0}

### Fetch Transactions

Specify constraints on transactions

In [44]:
Transaction.objects

[<Transaction: <Tr 2024-01-02 yoyodyne -> checking $5000.0 paycheck>>, <Transaction: <Tr 2024-01-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-02-02 yoyodyne -> checking $5000.0 paycheck>>, <Transaction: <Tr 2024-02-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-01-05 checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-02-05 checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-01-10 visa -> car $50.0 Shell Oil>>, <Transaction: <Tr 2024-02-26 visa -> car $60.0 Shell Oil>>, <Transaction: <Tr 2024-01-04 checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-02-04 checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway

In [45]:
Transaction.objects(description='Safeway')

[<Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway>>]

In [46]:
for t in Transaction.objects(description='Safeway'):
    for e in t.entries:
        print(e.date, e.account, e.amount, e.column)

2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit


In [47]:
for t in Transaction.objects(description='Safeway'):
    print(t.accounts)

{'checking', 'groceries'}
{'visa', 'groceries'}
{'checking', 'groceries'}
{'visa', 'groceries'}


In [48]:
for t in Transaction.objects(description='Safeway'):
    print(t.pamount)

75.0
175.0
75.0
75.0


In [49]:
for t in Transaction.objects(description='Safeway'):
    print(t.pdate, type(t.pdate))

2024-01-07 <class 'datetime.date'>
2024-01-21 <class 'datetime.date'>
2024-02-07 <class 'datetime.date'>
2024-02-21 <class 'datetime.date'>


In [50]:
for t in Transaction.objects(description='Safeway'):
    print(t.originals)

/
big party this weekend/
/
/


In [51]:
lst = list(Transaction.objects(description='Safeway'))

In [52]:
lst[1].comment

In [53]:
lst[1].pamount

175.0

In [54]:
lst[1].pdate

datetime.date(2024, 1, 21)

In [55]:
list(Transaction.objects(pamount__lt=175.0))

[<Transaction: <Tr 2024-01-10 visa -> car $50.0 Shell Oil>>,
 <Transaction: <Tr 2024-02-26 visa -> car $60.0 Shell Oil>>,
 <Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-03-10 amex -> household $50.0 Home Depot>>,
 <Transaction: <Tr 2024-03-12 household -> amex $25.0 Home Depot>>]

In [56]:
for t in Transaction.objects(pdate=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 checking yoyodyne
2024-01-02 5000.0 yoyodyne groceries/mortgage/car/travel


In [57]:
for t in Transaction.objects(pdate__lte=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 checking yoyodyne
2024-01-02 5000.0 yoyodyne groceries/mortgage/car/travel


### Operators

In [58]:
for t in Transaction.objects(description__gte='Safeway'):
    print(t.pdate, t.description)

2024-01-02 paycheck
2024-01-02 fill buckets
2024-02-02 paycheck
2024-02-02 fill buckets
2024-01-05 car payment
2024-02-05 car payment
2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [59]:
for t in Transaction.objects(description__regex='^S'):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


The operator automatically applies to list elements.

In [60]:
for t in Transaction.objects(description__regex=r'\s'):
    print(t.pdate, t.description, t.pamount)

2024-01-02 fill buckets 5000.0
2024-02-02 fill buckets 5000.0
2024-01-05 car payment 500.0
2024-02-05 car payment 500.0
2024-01-10 Shell Oil 50.0
2024-02-26 Shell Oil 60.0
2024-01-04 Rocket Mortgage 1800.0
2024-02-04 Rocket Mortgage 1800.0
2024-03-10 Home Depot 50.0
2024-03-12 Home Depot 25.0


For compound constraints we need another class from MongoEngine.

In [61]:
from mongoengine.queryset.visitor import Q

In [62]:
for t in Transaction.objects(Q(description__regex=r'^S')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [63]:
for t in Transaction.objects(Q(description__regex=r'^S') & Q(description__regex=r'\s')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


### QuerySet

In [64]:
for a in Account.nominal_accounts:
    print(a.name)

amex
visa
groceries
household
mortgage
car
travel


### Combining Query Elements

In [65]:
q = Q(description__regex=r'^S')

In [66]:
q

Q(**{'description__regex': '^S'})

In [67]:
type(q)

mongoengine.queryset.visitor.Q

In [68]:
p = Q(description__regex=r'\s')

In [69]:
p & q

(Q(**{'description__regex': '\\s'}) & Q(**{'description__regex': '^S'}))

In [70]:
for t in Transaction.objects(p & q):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


Create Q object using dictionaries

In [71]:
dct = {'description__regex': r'^S'}

In [72]:
Q(**dct)

Q(**{'description__regex': '^S'})

Can an object have multiple constraints?

In [73]:
dct = {'description__regex': r'^S', 'pamount__gt': 100}

In [74]:
Q(**dct)

Q(**{'description__regex': '^S', 'pamount__gt': 100})

In [75]:
for t in Transaction.objects(Q(**dct)):
    print(t.pdate, t.description, t.pamount)

2024-01-21 Safeway 175.0


Yep!

### Select Method

#### Select Transactions

All transactions:

In [76]:
for t in DB.select(Transaction):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 yoyodyne checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 checking car
2024-02-05 500.0 checking car
2024-01-10 50.0 visa car
2024-02-26 60.0 visa car
2024-01-04 1800.0 checking mortgage
2024-02-04 1800.0 checking mortgage
2024-01-07 75.0 checking groceries
2024-01-21 175.0 visa groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries
2024-03-10 50.0 amex household
2024-03-12 25.0 household amex


By date:

In [77]:
for t in DB.select(Transaction, date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-21 175.0 visa groceries


In [78]:
for t in DB.select(Transaction, start_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-02-02 5000.0 yoyodyne checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-05 500.0 checking car
2024-02-26 60.0 visa car
2024-02-04 1800.0 checking mortgage
2024-01-21 175.0 visa groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries
2024-03-10 50.0 amex household
2024-03-12 25.0 household amex


In [79]:
for t in DB.select(Transaction, end_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 checking car
2024-01-10 50.0 visa car
2024-01-04 1800.0 checking mortgage
2024-01-07 75.0 checking groceries
2024-01-21 175.0 visa groceries


By amount:

In [80]:
for t in DB.select(Transaction, amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-07 75.0 checking groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries


In [81]:
lst = DB.select(Transaction, amount=75)

In [82]:
all(t.pamount == 75 for t in lst)

True

In [83]:
for t in DB.select(Transaction, max_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-10 50.0 visa car
2024-02-26 60.0 visa car
2024-01-07 75.0 checking groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries
2024-03-10 50.0 amex household
2024-03-12 25.0 household amex


In [84]:
for t in DB.select(Transaction, min_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 yoyodyne checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 checking car
2024-02-05 500.0 checking car
2024-01-04 1800.0 checking mortgage
2024-02-04 1800.0 checking mortgage
2024-01-07 75.0 checking groceries
2024-01-21 175.0 visa groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries


By description:

In [85]:
for t in DB.select(Transaction, description = r'^s'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-10 50.0 Shell Oil visa car
2024-02-26 60.0 Shell Oil visa car
2024-01-07 75.0 Safeway checking groceries
2024-01-21 175.0 Safeway visa groceries
2024-02-07 75.0 Safeway checking groceries
2024-02-21 75.0 Safeway visa groceries


In [86]:
for t in DB.select(Transaction, comment=r'budget'):
    print(t.pdate, t.pamount, t.description, t.comment, t.pcredit, t.pdebit)

2024-01-02 5000.0 fill buckets 2024 budget groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 fill buckets 2024 budget groceries/mortgage/car/travel yoyodyne


By account:

In [87]:
for t in DB.select(Transaction, debit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-04 1800.0 checking mortgage
2024-02-04 1800.0 checking mortgage


In [88]:
for t in DB.select(Transaction, credit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne


Some random combinations

In [89]:
for t in DB.select(Transaction, description = r'^s', min_amount=100):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-21 175.0 Safeway visa groceries


In [90]:
for t in DB.select(Transaction, start_date = date(2024,2,1), credit='visa'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-02-26 60.0 Shell Oil visa car
2024-02-21 75.0 Safeway visa groceries


#### Select Entries

All entries:

In [91]:
len(DB.select(Entry))

38

In [92]:
for e in DB.select(Entry):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 visa 60.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 checking 1800.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 checking 1800.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0

By date:

In [93]:
for e in DB.select(Entry, date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit


In [94]:
for e in DB.select(Entry, start_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 visa 60.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 checking 1800.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit
2024-03-10 household 50.0 debit
2024-03-10 amex 50.0 credit
2024-03-12 household 25.0 credit
2024-03-12 amex 25.0 debit


In [95]:
for e in DB.select(Entry, end_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 checking 1800.0 credit


By amount:

In [96]:
for e in DB.select(Entry, amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 car 900.0 credit
2024-02-02 car 900.0 credit


In [97]:
for e in DB.select(Entry, max_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 groceries 300.0 credit
2024-01-02 car 900.0 credit
2024-02-02 groceries 300.0 credit
2024-02-02 car 900.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 visa 60.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit
2024-03-10 household 50.0 debit
2024-03-10 amex 50.0 credit
2024-03-12 household 25.0 credit
2024-03-12 amex 25.0 debit


In [98]:
for e in DB.select(Entry, min_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 checking 1800.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 checking 1800.0 credit


By account:

In [99]:
for e in DB.select(Entry, account='groceries'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 groceries 300.0 credit
2024-02-02 groceries 300.0 credit
2024-01-07 groceries 75.0 debit
2024-01-21 groceries 175.0 debit
2024-02-07 groceries 75.0 debit
2024-02-21 groceries 75.0 debit


By column:

In [100]:
for e in DB.select(Entry, column='credit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 yoyodyne 5000.0 credit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 checking 500.0 credit
2024-02-05 checking 500.0 credit
2024-01-10 visa 50.0 credit
2024-02-26 visa 60.0 credit
2024-01-04 checking 1800.0 credit
2024-02-04 checking 1800.0 credit
2024-01-07 checking 75.0 credit
2024-01-21 visa 175.0 credit
2024-02-07 checking 75.0 credit
2024-02-21 visa 75.0 credit
2024-03-10 amex 50.0 credit
2024-03-12 household 25.0 credit


In [101]:
for e in DB.select(Entry, column='debit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 debit
2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 debit
2024-01-05 car 500.0 debit
2024-02-05 car 500.0 debit
2024-01-10 car 50.0 debit
2024-02-26 car 60.0 debit
2024-01-04 mortgage 1800.0 debit
2024-02-04 mortgage 1800.0 debit
2024-01-07 groceries 75.0 debit
2024-01-21 groceries 175.0 debit
2024-02-07 groceries 75.0 debit
2024-02-21 groceries 75.0 debit
2024-03-10 household 50.0 debit
2024-03-12 amex 25.0 debit


### Serializing Objects

In [102]:
import json
from bson.objectid import ObjectId
import datetime

In [103]:
lst = DB.select(Transaction, start_date = date(2024,2,1), credit='visa')

In [104]:
lst[0].to_json()

'{"_id": {"$oid": "67e41dd9a0fded77104fa354"}, "description": "Shell Oil", "comment": "", "tags": [], "entries": [{"$oid": "67e41dd9a0fded77104fa33b"}, {"$oid": "67e41dd9a0fded77104fa33c"}], "pdate": {"$date": 1708905600000}, "pdebit": "car", "pcredit": "visa", "pamount": 60.0}'

In [105]:
type(lst[0])

dexter.DB.Transaction

In [106]:
obj = Transaction.objects.as_pymongo()[0]

In [107]:
type(obj)

dict

In [108]:
obj

{'_id': ObjectId('67e41dd9a0fded77104fa34d'),
 'description': 'paycheck',
 'comment': '',
 'tags': [],
 'entries': [ObjectId('67e41dd9a0fded77104fa327'),
  ObjectId('67e41dd9a0fded77104fa328')],
 'pdate': datetime.datetime(2024, 1, 2, 0, 0),
 'pdebit': 'checking',
 'pcredit': 'yoyodyne',
 'pamount': 5000.0}

In [109]:
s = lst[0].to_json()

In [110]:
json.loads(s)

{'_id': {'$oid': '67e41dd9a0fded77104fa354'},
 'description': 'Shell Oil',
 'comment': '',
 'tags': [],
 'entries': [{'$oid': '67e41dd9a0fded77104fa33b'},
  {'$oid': '67e41dd9a0fded77104fa33c'}],
 'pdate': {'$date': 1708905600000},
 'pdebit': 'car',
 'pcredit': 'visa',
 'pamount': 60.0}

In [111]:
Transaction.from_json(s)

<Transaction: <Tr 2024-02-26 visa -> car $60.0 Shell Oil>>

In [112]:
s = 'account: {...:...}'

In [113]:
s.find(':')

7

In [114]:
s[:s.find(':')]

'account'

In [115]:
s[s.find(':'):]

': {...:...}'

### Indexes

We want a field in Entry documents that serves as a unique ID so we can tell if an item was imported before.

MongoEngine has a UUID field.
* how is it computed?  is it a hash of all the other field values?
* when is it computed?  when the object is made, or when it is saved?

In [116]:
from mongoengine import *

In [117]:
class Foo(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)

In [118]:
f = Foo(name='Fred', amount=10)

Just declaring it is not enough to give it a value:

In [119]:
f.uid is None

True

In [120]:
f.save()

<Foo: Foo object>

This model has an index.  The `#` means it's a "hashed index" but no discussion of what that means or why we'd want one (over say a text index that we compute ourselves).

In [121]:
class Bar(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)
    meta = {
        'indexes': ['#uid']
    }

In [122]:
b1 = Bar(name='george', amount=20, uid='123')

In [123]:
b1.uid

'123'

Ah -- the UUID is created when the document is saved (the same was true for Foo, above, if a `uid` value passed to the constructor).

```
b1.save()
...
ValidationError: ValidationError (Bar:None) (Could not convert to UUID: badly formed hexadecimal UUID string: ['uid'])
```

So how do we make a UUID?  Do we care?  Why not just use our hashed strings?

#### UUID

It's in the PyMongo docs (and we had to specify how they're represented when we made the DB connection).  It's also a Python library.

In [124]:
from uuid import uuid4

In [125]:
b2 = Bar(name='ringo', amount=30, uid=uuid4())

In [126]:
b2.uid

UUID('2946604f-6b2c-46d5-8248-4058a99a9c34')

In [127]:
b2.save()

<Bar: Bar object>

In [128]:
x = b2.uid

In [129]:
b3 = Bar(name='paul', amount=40, uid=x)

In [130]:
b3.save()

<Bar: Bar object>

In [131]:
b3.uid == b2.uid

True

In [132]:
uuid4()

UUID('c0c2442c-b77c-4d3a-b380-e5d3b4c5cfd7')

So just defining an index isn't enough to make it unique.

### Unique

In [133]:
class FooBar(Document):
    name = StringField()
    amount = FloatField()
    uid = StringField(unique=True)

In [134]:
f1 = FooBar(name='Fred', amount=100, uid='bedrock')

In [135]:
f2 = FooBar(name='Barney', amount=200, uid='bedrock')

In [136]:
f1.save()

NotUniqueError: Tried to save duplicate unique keys (E11000 duplicate key error collection: pytest.foo_bar index: uid_1 dup key: { uid: "bedrock" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: pytest.foo_bar index: uid_1 dup key: { uid: "bedrock" }', 'keyPattern': {'uid': 1}, 'keyValue': {'uid': 'bedrock'}})

In [ ]:
try:
    f2.save()
except NotUniqueError as err:
    print(err)

In [ ]:
FooBar._meta

Awesome!  Just defining a field as unique is enough to have MongoEngine create an index.  Don't know (and don't care, yet, at least) about the ramifications of `sparse = False`.

> checked PyMongo docs, it's not what we think, and not something we want (even though we can have it by specifying `sparse=True` in the column spec)